In [30]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as pyplot
%matplotlib inline

In [31]:
# read in all of the words
words = open('names.txt','r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [32]:
len(words)

32033

In [33]:
#build the vocabulary of charactes and mappins to/from integers
chars = sorted(list(set(''.join(words))))
stoi ={s:i+1 for i, s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items()} #reverse stoi


In [34]:
#build the dataset
block_size = 3 #context: length how many characters do we take to predict the next one?
X, Y = [],[]
for w in words[:5]:
    print(w)
    context = [0]*block_size
    print(context)
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i]for i in context),'---->', itos[ix])
        context = context[1:] + [ix] # crop and append
        
X = torch.tensor(X)
Y = torch.tensor(Y)
        

emma
[0, 0, 0]
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
[0, 0, 0]
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
[0, 0, 0]
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
[0, 0, 0]
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
[0, 0, 0]
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [35]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [36]:

X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [37]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [59]:
#let's write a neural network that takes the Xs and outputs the Ys
C = torch.randn((27,2))
C

tensor([[ 0.8860,  0.1235],
        [ 0.4097, -0.3933],
        [ 1.8511, -1.3607],
        [-0.4307,  0.3019],
        [-0.1573,  0.0125],
        [ 0.4519, -0.4123],
        [ 0.9382,  0.5738],
        [ 1.5117, -0.3975],
        [-0.3228, -1.0485],
        [-0.3574,  0.9716],
        [ 0.3338,  0.8884],
        [-0.0319, -1.3790],
        [ 0.1163,  1.2368],
        [-1.5970,  0.8214],
        [ 0.3978,  2.5507],
        [-0.3844, -0.4150],
        [ 0.5275, -0.6594],
        [-0.3849,  1.6141],
        [ 0.9731,  0.6908],
        [ 0.9909,  1.0603],
        [ 1.3429,  1.8795],
        [ 0.4021, -0.8412],
        [ 2.6658, -0.4823],
        [-0.5356, -0.3142],
        [-0.9211,  0.5405],
        [ 0.7162, -0.1113],
        [-0.1094,  0.6603]])

In [60]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [79]:
emb = C[X]
emb.shape


torch.Size([32, 3, 2])

In [80]:
W1 = torch.randn((6,100))
b1 = torch.randn((100))

In [72]:
emb.shape


torch.Size([32, 3, 2])

In [88]:
h = torch.tanh(emb.view(-1,6)@ W1 + b1)

In [89]:
h

tensor([[-0.4542, -0.9929,  0.4790,  ..., -0.9016, -0.5618, -0.8361],
        [-0.7428, -0.8725,  0.7337,  ..., -0.4082, -0.8861, -0.8296],
        [ 0.9869, -0.9925, -0.9981,  ...,  0.9997, -0.9599,  0.5977],
        ...,
        [-0.9950,  0.4166,  0.7016,  ...,  0.9868, -0.9982,  0.6643],
        [ 0.7869, -0.9899, -0.9946,  ...,  0.7550, -0.9752,  0.9550],
        [-0.9898,  0.9766,  0.5139,  ..., -0.3469, -0.9949, -0.7972]])

In [90]:
h.shape

torch.Size([32, 100])

In [93]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [94]:
logits = h @ W2 + b2

In [95]:
logits.shape

torch.Size([32, 27])

In [96]:
counts = logits.exp()
prob = counts/counts.sum(1,keepdims=True)


In [97]:
prob.shape

torch.Size([32, 27])

In [100]:
loss = -prob[torch.arange(32),Y].log().mean()
loss

tensor(17.4298)

In [98]:
Y #the actual letter that comes next

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])